In [ ]:
import pandas as pd
import urllib
import random
import csv
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True) #Donne le chemin de notre Drive

#Versions:

#1.0 : Première version de l'algo avec uniquement la première page de chaque animé par nom
#1.1 : Simplification algorithmique du programme en enlevant les pages baissant ainsi la complexité
#1.2.0 : Ajout de la récupération du titre
#1.2.1 : Ajout de la récupération des genres
#1.2.3 : Ajout de la récupération des notes
#1.3.0 : connexion au drive du projet
#1.3.1 : Remplissage du dataset avec les données
#1.4 : Création du fichier csv contenant les informations
#1.5 : Simplification générale du programme et de la complexité

#2.0 : Conception de l'algorithme de recommandation de type KNN
#2.1 : Ajout de fonctions permettant d'adapter les informations de la base de donnée au fonctionnement de KNN
#2.2 : Fonctionnalité permettant de calculer la distance entre deux animés
#2.3 : Récuparation de la note des animés les plus proches
#2.4 : Affichage de la recommandation en %

Mounted at /content/drive


In [ ]:
#ALGORITHME DE SCRAPPING

url_site = "https://voiranime.com"

url_anime_by_first_letter = "https://voiranime.com/liste-danimes/page/"

anime_by_first_letter = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
anime_by_first_letter.insert(0,"non-char")
list_of_genres = ["Action","Adventure","Chinese","Comedy","Drama","Ecchi",
                 "Fantasy","Horror","Mahou Shoujo","Mecha","Music","Mystery"
                 "Psychological","Romance","Sci-Fi","Slice of Life","Sports",
                 "Supernatural","Thriller"]

genres_of_anime = ""
numeric_genres_of_anime = "a"
#print(len(genres_of_anime))

liste_titre_anime = []
liste_lien_anime = []
liste_note_anime = []
liste_genre_anime = []
liste_numeric_genre_anime = []

print("[Scrapping incoming]")

for letter in anime_by_first_letter:
  print()
  print("[Processing {}]".format(letter))
  for page in range(1,30):
    url = url_anime_by_first_letter + str(page) + "/??filter=subbed&start=" + str(letter)
    url_link = Request(url, headers={'User-Agent': 'Chrome/95.0.4638.54'})
    url_page = urlopen(url_link).read()
    url_html = bs(url_page, 'html.parser')
    containers = url_html.find_all('div',class_='post-title font-title')
    #print(containers)
    lol = str(url_html.find_all('div',class_='post-title font-title')) + "lol"
    #print(lol)
    if lol == "[]lol":
      break
    print()
    print("[page {}]".format(page))
    print()
 
    for items in containers:
      html_links = items.find_all('a')
      #print(html_links)
    
      for i,anime in enumerate(html_links):
      
        try:
          html_link = html_links[i].get('href') #getting url of anime
          #if html_link[-4:-1] == "-vf": #Permet de ne pas récupérer les animés en VF
            #break
          print(html_link)
          liste_lien_anime.append(html_link) #inclut l'url de l'anime dans le dataset
          anime_title = html_link[28:-1]
          liste_titre_anime.append(anime_title) #inclut le titre de l'anime dans le dataset

          anime_url_link = Request(html_link, headers={'User-Agent': 'Chrome/95.0.4638.54'})
          anime_url_page = urlopen(anime_url_link).read()
          anime_url_html = bs(anime_url_page, 'html.parser')

          ###

          anime_genres = anime_url_html.find_all('div',class_='genres-content')

          for genres in anime_genres:
            genres.find_all('a')
            genres = str(genres)
            #print(genres)
            for genre in list_of_genres:
              if genre in genres:
                #print(genre)
                numeric_genres_of_anime += "1"
                genres_of_anime += "&{}".format(genre)
              else:
                numeric_genres_of_anime += "0"
                
            genres_of_anime = genres_of_anime[1:]
            #print(genres_of_anime)   
            liste_genre_anime.append(genres_of_anime)
            liste_numeric_genre_anime.append(numeric_genres_of_anime)
            #print(numeric_genres_of_anime)
            genres_of_anime = ""
            numeric_genres_of_anime = "a"

            ###

            summary_of_animes = anime_url_html.find_all('div',class_='summary-content')
            test = str(summary_of_animes)
            #print(test)

            pos = [i for i in range(len(test)) if "<span id" == test[i:i+8]]
            #print(pos)
            note = test[pos[0]+24:pos[0]+27]
            if "</" in note:
              note = note[0]
            note = float(note)
            liste_note_anime.append(note) #inclut la note de l'anime dans le dataset
            #print(note)

        except Exception:
          print("error lol")
          pass
              

data = {'Titre':liste_titre_anime,'Lien':liste_lien_anime,'Note':liste_note_anime,'Genre':liste_genre_anime,'Genre numerique':liste_numeric_genre_anime}
df = pd.DataFrame(data)
df
df.to_csv(r'/content/drive/MyDrive/IA_Projet/Système de recommandation/voiranim_data_base.csv', index = False, header=True)

In [ ]:
#ALGORITHME DE RECOMMANDATION KNN

ANIME = input("Entrer nom de l'animé : ") #Format xxx-xxx-xxx
print()

data = pd.read_csv(r'/content/drive/MyDrive/IA_Projet/Système de recommandation/voiranim_data_base.csv')

def format_anime_name(nom_anime:str):
  nom_anime = nom_anime.lower()
  nom_anime = nom_anime.replace(" ","-")
  return(nom_anime)

def string_to_list(string:str):
  liste = list(string)
  liste.pop(0)
  for k in range(len(liste)):
    liste[k] = int(liste[k])
  return(liste)

def index_of_anime(nom_anime:str):
  try:
    return(int(data[data["Titre"] == nom_anime].index.values))
  except:
    return("invalid anime name")

def KKN_algorithme(nom_anime:str):
  try:
    nom_anime = format_anime_name(nom_anime)
    index = index_of_anime(nom_anime)
    all_nom_anime = list(data["Titre"])
    #print(all_nom_anime[index])
    liste_distance_anime = []
    liste_top_anime = []
    top_five_anime_note = []

    for k in range(len(all_nom_anime)):
      if k != index:
        test = string_to_list(data.loc[index,"Genre numerique"])
        temp = string_to_list(data.loc[k,"Genre numerique"])
        liste = []

        for i in range(len(temp)):
          liste.append(test[i] - temp[i])

        #print(liste)
        distance = 0
        for j in range(len(liste)):
          distance += pow(liste[j],2)
        distance = pow(distance,0.5)

        liste_distance_anime.append(distance)
        #print(distance)
        for l in range(len(liste_distance_anime)):
          if type(liste_distance_anime[l]) != float:
            print(liste_distance_anime[l])

    #print(liste_distance_anime)
    index_top_anime = [i for i in range(len(liste_distance_anime)) if 0 <= liste_distance_anime[i] <= 1.0]
    random.shuffle(index_top_anime) #Mélange la liste des index pour diversifier les retours d'une meme recommandation
    #print(index_top_five_anime)
    for m in index_top_anime:
      liste_top_anime.append(all_nom_anime[m])
      liste_top_five_anime = liste_top_anime[:5]
    #print(liste_top_five_anime)
    #print("{}".format(nom_anime) in liste_top_five_anime)

    for n in liste_top_five_anime:
      top_five_anime_note.append(data.loc[data['Titre'] == n, 'Note'].values[0])
    #print(top_five_anime_note)

    for o in range(5):
      print("{} est recommandé à {}%".format(liste_top_five_anime[o],int(top_five_anime_note[o]*20)))

  except:
    print("Nom de l'animé mal écrit ou non présent sur VOIRANIM")

KKN_algorithme(ANIME)

Entrer nom de l'animé : OnE PiecE

shin-tennis-no-ouji-sama-ova-vs-genius-10 est recommandé à 80%
one-piece-episode-of-sabo-3-kyoudai-no-kizuna-kiseki-no-saikai-to-uketsugareru-ishi est recommandé à 76%
one-piece-episode-of-merry-mou-hitori-no-nakama-no-monogatari est recommandé à 84%
fairy-tail est recommandé à 84%
lupin-the-iiird-chikemuri-no-ishikawa-goemon est recommandé à 76%
